# Gradient Descent Practice

In this repository, you are expecting to see the following analyses from scratch:

* __Model 1__: use a stat package in python & regularization <br/>
* __Model 2__: write gradient descent from scratch  <br/>
* __Model 3__: add regularization in model 2 <br/>
* __Model 4__: stochastic gradient descent (later)  <br/>
* __Model 5__: mini-batch gradient descent (later)  <br/>

### 1. Select the dataset: iris dataset

In [25]:
# import packages
import random
import math
import pandas as pd
import numpy as np
from sklearn import datasets

In [26]:
# read in the sample dataset from sklearn
iris = datasets.load_iris()
dat = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
# for simplicity, we only examine binary classification:
# map to whether not 1
value_map = {0. : 1, 
             1. : 0, 
             2. : 1} 
# replace the value in target & change column names
dat['target'] = dat['target'].map(value_map)
dat.columns = [var.replace(' (cm)', '') for var in dat.columns if '(cm)' in var] + ['target']
dat.head(3)

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1


In [27]:
dat.drop(['petal length', 'petal width'], axis = 1, inplace = True)

In [28]:
dat.columns

Index(['sepal length', 'sepal width', 'target'], dtype='object')

### Model 1: Use statistical packages

In [29]:
# Package 1: statsmodels.api
import statsmodels.api as sm
dat1 = dat.copy()
dat1['intercept'] = 1 # Note, if we don't have the intercept, statsmodels will by default has not intercept
model = sm.Logit(dat1['target'], dat1.loc[:,dat1.columns!='target'])
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.506818
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 target   No. Observations:                  150
Model:                          Logit   Df Residuals:                      147
Method:                           MLE   Df Model:                            2
Date:                Sun, 15 Jul 2018   Pseudo R-squ.:                  0.2038
Time:                        13:58:26   Log-Likelihood:                -76.023
converged:                       True   LL-Null:                       -95.477
                                        LLR p-value:                 3.557e-09
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
sepal length    -0.1404      0.246     -0.570      0.569      -0.623       0.342
sepal width      3.2142      0.642      5.010      0.000       1.957       4.472
intercept       -8.0251      2.387     -3.362      0.001     -12.703      -3.347
================================================================================
"""

In [30]:
# Package 2: sklearn
from sklearn.linear_model import LogisticRegression
dat2 = dat.copy()
lr = LogisticRegression(random_state=0, C=1e5) # lower C means higher penalty, use convention in SVM
lr.fit(dat2.loc[:,dat2.columns!='target'], dat2['target'])
print('intercept: ', lr.intercept_)
dict(zip(dat2.columns[dat2.columns!='target'], lr.coef_[0]))

intercept:  [-8.02399007]


{'sepal length': -0.1404119881410533, 'sepal width': 3.213924585692371}

In [50]:
# sklearn with l2 regularization, C=25
lr = LogisticRegression(random_state=0, C=25, penalty='l2') # lower C means higher penalty, use convention in SVM
lr.fit(dat2.loc[:,dat2.columns!='target'], dat2['target'])
print('intercept: ', lr.intercept_)
dict(zip(dat2.columns[dat2.columns!='target'], lr.coef_[0]))

intercept:  [-6.49918279]


{'sepal length': -0.230651632407577, 'sepal width': 2.87903841342581}

We can see the result is slightly different, probably because one requires smaller different between iterations to determine the convergence. Therefore, we believe everything is correct here. 

### Model 2: Build the logistic regression from scratch

In [31]:
# review the dataset
dat.describe()

,sepal length,sepal width,target
count,150.000000,150.000000,150.000000
mean,5.843333,3.054000,0.666667
std,0.828066,0.433594,0.472984
min,4.300000,2.000000,0.000000
25%,5.100000,2.800000,0.000000
50%,5.800000,3.000000,1.000000
75%,6.400000,3.300000,1.000000
max,7.900000,4.400000,1.000000


In [32]:
# build sigmoid functions
def sigmoid(x):
    return(1/(1+math.exp(-x))) # we should raise exception for different x data type

In [33]:
# build loss function 
def lr_loss(y=1, p=0.5):
    if (y in [0,1]) and (0<p<1):
        return(-y*math.log(p)-(1-y)*math.log(1-p))
    else:
        raise ValueError('input y or p is out of bound.')

In [34]:
# let's train the model
def lr_train(data = dat, fit_intercept = True, random_state = 0, alpha = [0.05, 1, 1], tol = 1e-5, target = 'target', varList = []):
    # assume there is no column called 'intercept'
    if fit_intercept:
        dat['intercept'] = 1
        varList.append('intercept')
    # initiate beta based on random_state:
    random.seed(random_state) 
    init_beta = [0] * 3
    new_beta = [random.random() for i in range(len(varList))]
    # new_beta = [0.26,  2.779, -1.2988, 2.703, -7.320] 
    # add two columns: predicted prob 
    data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], init_beta)), axis = 1)
    data['loss'] = data.apply(lambda row: lr_loss(y=row['target'], p=row['pred']), axis=1)
    # loop through
    while max(abs(np.array(new_beta) - np.array(init_beta)))>tol:
        for i in range(len(varList)): 
            init_beta[i] = new_beta[i]
            new_beta[i] = init_beta[i] + alpha[i]*(1/data.shape[0])*sum((data['target']-data['pred'])*data[varList[i]])
        data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], new_beta)), axis = 1)
        print('updated:', new_beta)
    return(new_beta)

In [35]:
dat.columns

Index(['sepal length', 'sepal width', 'target'], dtype='object')

In [36]:
print(lr_train(data=dat, varList = ['sepal length', 'sepal width'])) # , 'petal length', 'petal width'

updated: [0.8915718515250481, 1.361621069606969, 0.5872382474975117]
updated: [0.7926570719970795, 0.43848290608001717, 0.25397398124180665]
updated: [0.6943733733761486, -0.4776175749097724, -0.0769501339509851]
updated: [0.618822905323893, -1.1282762398110373, -0.32426379369673497]
updated: [0.6698824689972585, -0.3756637391887241, -0.125963188974847]
updated: [0.591658107147301, -1.0594403066395173, -0.38328101510329304]
updated: [0.6436306207663489, -0.302132647995621, -0.18256533583195833]
updated: [0.5649076567046641, -0.993405156586585, -0.4419194652896595]
updated: [0.6181666121989673, -0.22753188233629085, -0.23752245600608055]
updated: [0.5388099289976852, -0.9277513238195896, -0.49939228890285936]
updated: [0.5931022783373607, -0.155945551538048, -0.2921720030700996]
updated: [0.5132712316895461, -0.8631654333899363, -0.5559657801410562]
updated: [0.5685315163909819, -0.08610735276441772, -0.3461434294702107]
updated: [0.48829132610767323, -0.7994857121301304, -0.61161373880

updated: [-0.19119325054835093, 1.9248020391484064, -2.343125072109931]
updated: [-0.259207115361448, 1.2899064582520925, -2.5727030444770236]
updated: [-0.1989158421604396, 1.9447088234170575, -2.3703092140299256]
updated: [-0.26645833711650585, 1.3138721721517017, -2.598367628014205]
updated: [-0.20644709214155652, 1.9641371141924406, -2.397152939008753]
updated: [-0.2735158452770775, 1.337385947409091, -2.6236835577471163]
updated: [-0.21379077819710895, 1.9830977230437854, -2.423661984966738]
updated: [-0.28038366415189453, 1.3604563099018288, -2.6486573871331833]
updated: [-0.22095061030802116, 2.0016012347706695, -2.4498419958627324]
updated: [-0.2870657462352004, 1.3830916017092718, -2.6732955607415]
updated: [-0.22793023241975846, 2.0196580138955387, -2.4756985237752787]
updated: [-0.2935659738029184, 1.4052999874857945, -2.6976044161913575]
updated: [-0.23473322407715896, 2.0372782108060687, -2.501237030956078]
updated: [-0.29988816044789957, 1.4270894606319589, -2.72159018603

updated: [-0.4584451735203259, 2.138657056012744, -3.6474614428636736]
updated: [-0.41738649133074124, 2.550550332176364, -3.5160475669456983]
updated: [-0.45934653876920273, 2.146606065986674, -3.66094956670003]
updated: [-0.4185863235978874, 2.55519229364025, -3.5305660565040866]
updated: [-0.46019890442793354, 2.1544089970907363, -3.6743280433397847]
updated: [-0.419734450889734, 2.5597253312593153, -3.544964331976691]
updated: [-0.4610035586785983, 2.16206886053896, -3.6875990219092754]
updated: [-0.4208321404032644, 2.564152384011367, -3.5592445160853545]
updated: [-0.4617617601000709, 2.169588611796033, -3.700764602908158]
updated: [-0.421880632286349, 2.5684763168294498, -3.573408690367765]
updated: [-0.462474738212311, 2.176971151346117, -3.713826839119048]
updated: [-0.4228811400885177, 2.5726999221519646, -3.5874588957944287]
updated: [-0.4631436940172045, 2.184219325459745, -3.7267877365112256]
updated: [-0.4238348512100955, 2.5768259214545717, -3.6013971333858117]
updated: 

updated: [-0.4310689196219638, 2.7052913463483037, -4.210155379036669]
updated: [-0.45868673945674, 2.4339273664810537, -4.307547033330429]
updated: [-0.43070557311143304, 2.7067540305356768, -4.220187861379862]
updated: [-0.4581600816312916, 2.436929544477068, -4.31703077336747]
updated: [-0.4303316689942688, 2.708195062013274, -4.230172762389433]
updated: [-0.4576249667782155, 2.4398911023532306, -4.326473736338591]
updated: [-0.42994752919851437, 2.709615021321723, -4.240110712416216]
updated: [-0.4570816814247082, 2.4428129307151702, -4.335876439531153]
updated: [-0.42955346768044134, 2.711014471909762, -4.250002328544485]
updated: [-0.45653050443695803, 2.4456959007562915, -4.345239387914229]
updated: [-0.42914979061054376, 2.71239396061, -4.259848214881672]
updated: [-0.4559717072132873, 2.4485408646543054, -4.35456307445273]
updated: [-0.4287367965555172, 2.713754018102576, -4.269648962842221]
updated: [-0.45540555387275583, 2.451348655960617, -4.36384798041397]
updated: [-0.428

updated: [-0.4204783477156458, 2.5617082806481486, -4.802110981294601]
updated: [-0.3991778838123831, 2.767553467456059, -4.736331182162654]
updated: [-0.4197629122112923, 2.5633668629717605, -4.809808966465686]
updated: [-0.39855083533390606, 2.76834278635053, -4.744310134719152]
updated: [-0.41904737923208996, 2.5650140123605287, -4.817482931765236]
updated: [-0.3979231287445154, 2.769126562558858, -4.752263188937411]
updated: [-0.41833180351887966, 2.5666499668479483, -4.825133022709747]
updated: [-0.39729483233165325, 2.7699048893926568, -4.760190536977393]
updated: [-0.41761623816798166, 2.5682749589919696, -4.83275938215482]
updated: [-0.39666601250843425, 2.7706778572635757, -4.768092367527436]
updated: [-0.416900734675723, 2.5698892159993796, -4.840362150365045]
updated: [-0.39603673386070226, 2.7714455537742864, -4.7759688658838355]
updated: [-0.41618534298181104, 2.5714929598475766, -4.847941465082036]
updated: [-0.3954070591929646, 2.7722080638067776, -4.783820214028652]
upd

...

updated: [-0.14147144637078526, 3.2110012704286417, -8.00905410711187]
updated: [-0.14147029022787172, 3.2110045911057257, -8.0090707811193]
updated: [-0.1414691352842394, 3.211007908342024, -8.009087437841497]
updated: [-0.1414679815386415, 3.211011222141112, -8.009104077296396]
updated: [-0.14146682898983404, 3.211014532506544, -8.009120699501917]
updated: [-0.1414656776365726, 3.2110178394418876, -8.00913730447596]
updated: [-0.14146452747761568, 3.211021142950693, -8.009153892236407]
updated: [-0.1414633785117218, 3.2110244430365182, -8.009170462801121]
updated: [-0.14146223073765182, 3.211027739702907, -8.00918701618795]
updated: [-0.14146108415416683, 3.21103103295341, -8.009203552414714]
updated: [-0.14145993876003038, 3.2110343227915625, -8.009220071499227]
updated: [-0.14145879455400612, 3.211037609220908, -8.009236573459274]
updated: [-0.14145765153486003, 3.2110408922449767, -8.009253058312627]
updated: [-0.14145650970135862, 3.2110441718673015, -8.009269526077038]
updated: 

updated: [-0.14134297726827078, 3.21137028302339, -8.010906968086426]
updated: [-0.1413419543750582, 3.2113732213538624, -8.010921721427536]
updated: [-0.14134093254269875, 3.211376156639934, -8.010936459476348]
updated: [-0.14133991177009061, 3.2113790888847595, -8.010951182248725]
updated: [-0.14133889205613306, 3.2113820180914927, -8.010965889760516]
updated: [-0.14133787339972662, 3.211384944263281, -8.010980582027557]
updated: [-0.1413368557997726, 3.2113878674032734, -8.01099525906566]
updated: [-0.141335839255174, 3.21139078751461, -8.01100992089063]
updated: [-0.14133482376483453, 3.211393704600433, -8.011024567518245]
updated: [-0.14133380932765935, 3.211396618663876, -8.011039198964275]
updated: [-0.14133279594255438, 3.2113995297080744, -8.01105381524447]
updated: [-0.14133178360842721, 3.2114024377361554, -8.011068416374565]
updated: [-0.1413307723241859, 3.2114053427512492, -8.011083002370277]
updated: [-0.14132976208874035, 3.2114082447564756, -8.011097573247309]
updated:

updated: [-0.14123021939679603, 3.211694205613178, -8.012533343172723]
updated: [-0.1412293134360501, 3.211696808337593, -8.012546410814439]
updated: [-0.14122840841472523, 3.211699408365487, -8.012559464912583]
updated: [-0.14122750433184586, 3.2117020056996557, -8.012572505481202]
updated: [-0.1412266011864377, 3.21170460034289, -8.012585532534331]
updated: [-0.1412256989775274, 3.2117071922979794, -8.01259854608599]
updated: [-0.14122479770414245, 3.211709781567711, -8.012611546150183]
updated: [-0.14122389736531157, 3.211712368154868, -8.0126245327409]
updated: [-0.14122299796006454, 3.2117149520622297, -8.012637505872116]
updated: [-0.14122209948743186, 3.2117175332925756, -8.012650465557792]
updated: [-0.1412212019464454, 3.211720111848678, -8.012663411811877]
updated: [-0.1412203053361376, 3.211722687733312, -8.012676344648302]
updated: [-0.14121940965554242, 3.2117252609492435, -8.012689264080985]
updated: [-0.14121851490369422, 3.2117278314992417, -8.012702170123827]
updated: 

updated: [-0.14113035052193826, 3.2119811318350484, -8.013973899284448]
updated: [-0.14112954811080583, 3.2119834373007845, -8.01398547397428]
updated: [-0.14112874653159482, 3.2119857403780943, -8.013997036669004]
updated: [-0.14112794578344118, 3.211988041069457, -8.014008587381056]
updated: [-0.1411271458654827, 3.2119903393773424, -8.014020126122865]
updated: [-0.14112634677685734, 3.211992635304223, -8.014031652906844]
updated: [-0.14112554851670459, 3.2119949288525635, -8.014043167745395]
updated: [-0.1411247510841641, 3.2119972200248332, -8.014054670650905]
updated: [-0.1411239544783773, 3.21199950882349, -8.014066161635752]
updated: [-0.14112315869848557, 3.2120017952509983, -8.014077640712294]
updated: [-0.14112236374363213, 3.212004079309811, -8.014089107892886]
updated: [-0.14112156961296013, 3.212006361002388, -8.014100563189862]
updated: [-0.14112077630561448, 3.2120086403311774, -8.014112006615546]
updated: [-0.14111998382074037, 3.2120109172986324, -8.014123438182251]
up

updated: [-0.14104189571372508, 3.2122352888005197, -8.01524988159535]
updated: [-0.141041185004388, 3.212237330969907, -8.015260134011067]
updated: [-0.14104047503179887, 3.212239371023742, -8.015270375802865]
updated: [-0.14103976579519364, 3.2122414089642106, -8.015280606981763]
updated: [-0.14103905729380808, 3.2122434447935095, -8.015290827558763]
updated: [-0.1410383495268798, 3.2122454785138204, -8.01530103754486]
updated: [-0.1410376424936461, 3.2122475101273347, -8.015311236951035]
updated: [-0.14103693619334617, 3.2122495396362294, -8.01532142578826]
updated: [-0.14103623062521883, 3.2122515670426925, -8.015331604067491]
updated: [-0.14103552578850498, 3.2122535923488957, -8.015341771799678]
updated: [-0.14103482168244494, 3.212255615557022, -8.015351928995754]
updated: [-0.1410341183062812, 3.2122576366692392, -8.015362075666648]
updated: [-0.14103341565925573, 3.212259655687726, -8.015372211823271]
updated: [-0.14103271374061238, 3.212261672614646, -8.015382337476526]
updat

There is something wrong with this chunk of code:
    1. it could be parallel computed
    2. the running time is really volatile -> could be something wrong with the code here (*)

the result <br>
[-0.14102365473816966, 3.2122877033154436, -8.01551302010229]
<br>
It just takes too long to compute. The learning rate should be selected really carefully. 

### Model 3: add regularization in model 2

In [46]:
def lr_train_l2reg(data = dat, fit_intercept = True, C = 5e1, random_state = 0, alpha = [0.05, 1, 1], tol = 1e-5, target = 'target', varList = []):
    # assume there is no column called 'intercept'
    if fit_intercept:
        dat['intercept'] = 1
        varList.append('intercept')
    # initiate beta based on random_state:
    random.seed(random_state) 
    init_beta = [0] * 3
    new_beta = [random.random() for i in range(len(varList))]
    # new_beta = [0.26,  2.779, -1.2988, 2.703, -7.320] 
    # add two columns: predicted prob 
    data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], init_beta)), axis = 1)
    data['loss'] = data.apply(lambda row: lr_loss(y=row['target'], p=row['pred']), axis=1)
    # loop through
    while max(abs(np.array(new_beta) - np.array(init_beta)))>tol:
        for i in range(len(varList)): 
            init_beta[i] = new_beta[i]
            new_beta[i] = init_beta[i] + alpha[i]*(1/data.shape[0])*(sum((data['target']-data['pred'])*data[varList[i]])-(2/C)*init_beta[i])
        data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], new_beta)), axis = 1)
        print('updated:', new_beta)
    return(new_beta)

In [47]:
print(lr_train_l2reg(data=dat, varList = ['sepal length', 'sepal width'])) # , 'petal length', 'petal width'     

updated: [0.8915605925670278, 1.3614189484328516, 0.5871260950759568]
updated: [0.7926339393712684, 0.43791788768791984, 0.2537053132464086]
updated: [0.6943410258140756, -0.4782840223360837, -0.07728143332531168]
updated: [0.6188358780129078, -1.1281742777385917, -0.3243747817637971]
updated: [0.6698693753362286, -0.37544667688868216, -0.12604796144889222]
updated: [0.591626259638737, -1.0592395888930919, -0.38336825142117326]
updated: [0.6435693773128892, -0.30188060945702844, -0.18262433907569098]
updated: [0.5648307302444854, -0.9931589541528691, -0.4419561881773714]
updated: [0.6180658786584515, -0.22720386396206305, -0.2374987072275303]
updated: [0.5386923642270952, -0.9274672843090879, -0.49933726222034613]
updated: [0.592963591779053, -0.15557842091901375, -0.2920343585562647]
updated: [0.5131159416814242, -0.862863213755572, -0.5557828789095676]
updated: [0.5683582141560645, -0.08571514288137916, -0.34585448751179104]
updated: [0.48810144069358635, -0.7991807869822365, -0.6112

updated: [-0.19177075300515162, 1.9144267720536066, -2.3020906771246272]
updated: [-0.2599979710927264, 1.2770792235678416, -2.5317890231758606]
updated: [-0.19950724300008127, 1.9340425393013925, -2.328056124990286]
updated: [-0.26727391166304915, 1.3006503235802076, -2.556264700538876]
updated: [-0.20705335255946575, 1.9531804143979936, -2.3536716293050377]
updated: [-0.2743574185062908, 1.3237672870212762, -2.5803833438336374]
updated: [-0.21441287945081577, 1.971851327501669, -2.3789432017982235]
updated: [-0.28125253530299943, 1.3464387554868202, -2.6041517767117077]
updated: [-0.22158955249099285, 1.9900659766994366, -2.403876754090112]
updated: [-0.28796323244273814, 1.3686731837569095, -2.6275767070096436]
updated: [-0.22858703324533286, 2.007834834578965, -2.4284780998399476]
updated: [-0.29449340863935464, 1.3904788461855544, -2.6506647287824245]
updated: [-0.23540891767322292, 2.025168154453904, -2.4527529568660213]
updated: [-0.30084689248651614, 1.4118638428865689, -2.6734

updated: [-0.462889194038385, 2.1017179075109538, -3.5296303894382666]
updated: [-0.42061941110478634, 2.5259883713763776, -3.3933014839100077]
updated: [-0.46388868145799156, 2.1092554074214864, -3.5417225789609192]
updated: [-0.42189373945554814, 2.530452503518749, -3.406344075951073]
updated: [-0.46484008168871827, 2.116647394393879, -3.55370432297345]
updated: [-0.4231171304658756, 2.53480961135984, -3.4192652242792176]
updated: [-0.4657446673938595, 2.123896885172037, -3.5655778153807884]
updated: [-0.42429083884536517, 2.539062611620708, -3.432067105299268]
updated: [-0.4666036817388101, 2.1310068402777813, -3.5773452001200416]
updated: [-0.4254160923302024, 2.5432143467980315, -3.4447518528109895]
updated: [-0.4674183389435064, 2.13798016480949, -3.589008572118528]
updated: [-0.42649409214178724, 2.547267586743828, -3.4573215586737485]
updated: [-0.4681898248307868, 2.144819709237338, -3.6005699782441876]
updated: [-0.4275260134431231, 2.551225030225369, -3.4697782734697062]
upd

updated: [-0.4395563071221396, 2.6728798758224426, -4.000991853321241]
updated: [-0.4700448367944191, 2.3737372046311616, -4.10701607085597]
updated: [-0.4392950818664988, 2.6742399756956514, -4.009536254656347]
updated: [-0.46964857120345693, 2.3763631757212784, -4.115105255923529]
updated: [-0.4390235455516773, 2.675579204889784, -4.018033554813875]
updated: [-0.4692441116502005, 2.3789490892390193, -4.123154221915438]
updated: [-0.4387420144290416, 2.676898131369329, -4.02648439889781]
updated: [-0.4688317380608976, 2.381495832798645, -4.131163497520359]
updated: [-0.4384507968523813, 2.6781973062625846, -4.034889418441547]
updated: [-0.46841172281164906, 2.384004274497133, -4.139133598930334]
updated: [-0.43815019346190687, 2.679477264336687, -4.0432492317025295]
updated: [-0.46798433091962827, 2.3864752633090025, -4.147065030160026]
updated: [-0.437840497364274, 2.6807385244604776, -4.051564443950895]
updated: [-0.46754982022979286, 2.3889096294740697, -4.154958283358251]
updated:

updated: [-0.439669852769952, 2.4804876377979226, -4.52191236830168]
updated: [-0.41395355977350984, 2.730129232629883, -4.440554449877594]
updated: [-0.43909190238739293, 2.4817942248226887, -4.528263088347433]
updated: [-0.4134351976142053, 2.7308483219851354, -4.447090851403845]
updated: [-0.4385138714762906, 2.4830895874882537, -4.534590819219747]
updated: [-0.41291617150013044, 2.7315622999461366, -4.453602312067612]
updated: [-0.43793581232070683, 2.48437395519425, -4.540895712465994]
updated: [-0.412396546422216, 2.732271259482516, -4.4600890271968305]
updated: [-0.43735777560323325, 2.4856475518231806, -4.547177916925225]
updated: [-0.41187638552255146, 2.732975290794195, -4.466551188542949]
updated: [-0.4367798104488151, 2.4869105958673448, -4.553437578798525]
updated: [-0.41135575014133063, 2.7336744813991536, -4.472988984362923]
updated: [-0.43620196446742676, 2.4881633005531367, -4.5596748417174835]
updated: [-0.4108346998626792, 2.7343689162184948, -4.479402599499381]
upda

...

updated: [-0.2317753652841289, 2.8762739745666623, -6.484322262581614]
updated: [-0.23177353503372208, 2.8762804375521998, -6.484342073025755]
updated: [-0.2317722363703544, 2.8762816858720273, -6.484363623803425]
updated: [-0.23177041418547614, 2.8762881016524102, -6.484383389067355]
updated: [-0.23176911618742232, 2.876289375539197, -6.484404870035855]
updated: [-0.23176730200489942, 2.8762957446462356, -6.484424590109333]
updated: [-0.2317660047107077, 2.8762970436309976, -6.484446001598068]
updated: [-0.23176419846808946, 2.876303366589645, -6.484465676472707]
updated: [-0.2317629019156581, 2.876304690210046, -6.484487018808347]
updated: [-0.2317611035512072, 2.876310967538488, -6.4845066484775895]
updated: [-0.2317598077777934, 2.8763123153387524, -6.484527921984105]
updated: [-0.231758017230477, 2.876318547548495, -6.484547506443184]
updated: [-0.23175672227270597, 2.876319919079316, -6.484568711441876]
updated: [-0.2317549394821864, 2.8763261066752754, -6.484588250687797]
update

updated: [-0.2316061422660918, 2.876691742279791, -6.486557841693529]
updated: [-0.2316049291654365, 2.876693870519689, -6.486575815458737]
updated: [-0.23160348653263452, 2.8766982709990128, -6.4865929915244465]
updated: [-0.23160227549094348, 2.8767004062871693, -6.486610910893056]
updated: [-0.2316008381812082, 2.876704781658247, -6.486628043555294]
updated: [-0.23159962921105529, 2.8767069237947656, -6.486645908739908]
updated: [-0.23159819719136537, 2.8767112743064582, -6.486662998058674]
updated: [-0.231596990305039, 2.876713423094463, -6.486680809270468]
updated: [-0.2315955635427138, 2.8767177489924913, -6.486697855306425]
updated: [-0.23159435875222126, 2.876719904238088, -6.486715612755161]
updated: [-0.23159293721491692, 2.876724205765072, -6.486732615569624]
updated: [-0.23159173453198822, 2.8767263672773056, -6.4867503194636615]
updated: [-0.23159031818769327, 2.8767306446728065, -6.486767279118585]
updated: [-0.23158911762378542, 2.8767328122636218, -6.486784929664902]
up

updated: [-0.23146370423236848, 2.8770420156393683, -6.488442965068067]
updated: [-0.2314626240627842, 2.877044300158781, -6.4884580968313745]
updated: [-0.2314614445131524, 2.8770475741917254, -6.488472870406975]
updated: [-0.23146036676751092, 2.8770498580851087, -6.48848795814063]
updated: [-0.23145919107766547, 2.877053117331063, -6.488502692557406]
updated: [-0.2314581157574026, 2.877055400519552, -6.488517736411295]
updated: [-0.2314569439084613, 2.8770586450996696, -6.488532431751032]
updated: [-0.23145587101489515, 2.877060927505738, -6.4885474318742435]
updated: [-0.2314547029881414, 2.877064157539722, -6.4885620882188775]
updated: [-0.23145363252247428, 2.877066439087164, -6.488577044759711]
updated: [-0.23145246829935495, 2.87706965469329, -6.488591662191322]
updated: [-0.23145140026267544, 2.877071935307199, -6.488606575297293]
updated: [-0.23145023982479898, 2.877075136602332, -6.488621153898099]
updated: [-0.2314491742180835, 2.877077416209085, -6.488636023715949]
updated

updated: [-0.2313415688280253, 2.8773423452819618, -6.490059649983444]
updated: [-0.23134058630031748, 2.8773449243372533, -6.490072290138387]
updated: [-0.2313396485148522, 2.877347073001783, -6.490085057652388]
updated: [-0.23133866897335875, 2.8773496424002065, -6.490097663498733]
updated: [-0.2313377335460678, 2.877351787560239, -6.490110394667038]
updated: [-0.23133675697887965, 2.8773543473640197, -6.490122966288228]
updated: [-0.23133582390676521, 2.8773564889941707, -6.490135661223887]
updated: [-0.23133485030205703, 2.877359039264863, -6.490148198703299]
updated: [-0.2313339195820794, 2.877361177340314, -6.4901608575188785]
updated: [-0.23133294892810868, 2.877363718138811, -6.490173360939826]
updated: [-0.23133202055718732, 2.8773658526352976, -6.490185983747414]
updated: [-0.2313310528422934, 2.877368384021839, -6.490198453193142]
updated: [-0.23133012681730747, 2.877370514915647, -6.4902110401043505]
updated: [-0.23132916202991075, 2.8773730369498276, -6.490223475658035]
up

updated: [-0.23123775544487135, 2.8775979950062416, -6.4914331267427485]
updated: [-0.23123694848466134, 2.877599912778366, -6.4914439595732105]
updated: [-0.2312361241187085, 2.8776020102068043, -6.491454715411124]
updated: [-0.23123531929441754, 2.8776039237188304, -6.491465517752827]
updated: [-0.23123449732976945, 2.8776060142524402, -6.49147624404122]
updated: [-0.23123369463701396, 2.8776079235008356, -6.491487015984201]
updated: [-0.23123287506543494, 2.877610007174053, -6.491497712800092]
updated: [-0.23123207449982128, 2.8776119121555137, -6.491508454434063]
updated: [-0.23123125731312072, 2.877613989002462, -6.49151912185433]
updated: [-0.23123045887024557, 2.8776158897139084, -6.491529833268682]
updated: [-0.23122964406027732, 2.877617959768402, -6.491540471370063]
updated: [-0.23122884773572813, 2.8776198562069757, -6.491551152653863]
updated: [-0.23122803529439023, 2.8776219195025243, -6.491561761512951]
updated: [-0.23122724108374548, 2.877623811665585, -6.491572412754951

the result <br>
[-0.23119375274983178, 2.877706303593354, -6.492015452858157]
<br>
It just takes too long to compute. The learning rate should be selected really carefully. 